In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/My\ Drive/the_boys.zip

Archive:  /content/drive/My Drive/the_boys.zip
   creating: the_boys/
  inflating: the_boys/s01e01.csv     
  inflating: the_boys/s01e02.csv     
  inflating: the_boys/s01e03.csv     
  inflating: the_boys/s01e04.csv     
  inflating: the_boys/s01e05.csv     
  inflating: the_boys/s01e06.csv     
  inflating: the_boys/s01e07.csv     
  inflating: the_boys/s01e08.csv     
  inflating: the_boys/s02e01.csv     
  inflating: the_boys/s02e02.csv     
  inflating: the_boys/s02e03.csv     
  inflating: the_boys/s02e04.csv     
  inflating: the_boys/s02e05.csv     
  inflating: the_boys/s02e06.csv     
  inflating: the_boys/s02e07.csv     
  inflating: the_boys/s02e08.csv     
  inflating: the_boys/s03e01.csv     
  inflating: the_boys/s03e02.csv     
  inflating: the_boys/s03e03.csv     
  inflating: the_boys/s03e04.csv     
  inflating: the_boys/s03e05.csv     
  inflating: the_boys/s03e06.csv     
  inflating: the_boys/s03e07.csv     
  inflating: the_boys/s03e08.csv     


In [3]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
import csv as csv
import zipfile
import os
import matplotlib.pyplot as plt
import itertools 

zip_file_path = '/content/drive/MyDrive/the_boys.zip'

In [350]:
# Creating the empty dataframe to store all our data
the_boys = pd.DataFrame()

In [351]:
# Unzipping the csv files to create a combined dataframe

zf = zipfile.ZipFile(zip_file_path) 
zf.namelist()
zip_files = {file for file in zf.namelist() if file.endswith('.csv')}

for boy in zip_files:
    df = pd.read_csv(zf.open(boy))
    df['Season'] = boy.split('/')[1].strip('.csv')[1:2]
    df['Episode'] = boy.split('/')[1].strip('.csv')[4:5]
    the_boys = the_boys.append(df)

the_boys

,nconst,character,start,end,Season,Episode
0,nm6150071,Frenchie,70000,158000,2,6
1,nm7343952,Cherie,74000,158000,2,6
2,nm5216674,Jay,84000,158000,2,6
3,nm6150071,Frenchie,158000,243000,2,6
4,nm3929195,Starlight,159000,243000,2,6
...,...,...,...,...,...,...
152,nm6150071,Frenchie,3450000,3572000,3,7
153,nm7232332,Kimiko,3450000,3572000,3,7
154,nm1102278,Homelander,3572000,3599000,3,7
155,nm0010075,Soldier Boy,3599000,3657000,3,7


In [352]:
# Adding an additional column to calculate the length of a scene
the_boys['Scene Duration'] = the_boys['end'] - the_boys['start']
the_boys

,nconst,character,start,end,Season,Episode,Scene Duration
0,nm6150071,Frenchie,70000,158000,2,6,88000
1,nm7343952,Cherie,74000,158000,2,6,84000
2,nm5216674,Jay,84000,158000,2,6,74000
3,nm6150071,Frenchie,158000,243000,2,6,85000
4,nm3929195,Starlight,159000,243000,2,6,84000
...,...,...,...,...,...,...,...
152,nm6150071,Frenchie,3450000,3572000,3,7,122000
153,nm7232332,Kimiko,3450000,3572000,3,7,122000
154,nm1102278,Homelander,3572000,3599000,3,7,27000
155,nm0010075,Soldier Boy,3599000,3657000,3,7,58000


In [353]:
# How many seasons are there?
print("The Boys has {} seasons.".format((the_boys['Season'].nunique())))

The Boys has 3 seasons.


In [354]:
# How many episodes per season are there?
the_boys[['Season', 'Episode']].groupby('Season').nunique()

,Episode
Season,
1,8
2,8
3,8


In [355]:
# How many characters are there?
print("In total, {} characters appear in The Boys.".format((the_boys['nconst'].nunique())))

In total, 298 characters appear in The Boys.


In [356]:
character_names = the_boys['character'].unique()
character_ids = the_boys['nconst'].unique()

print(len(character_ids) == len(character_names))

# So we find out that the names are more than the unique identifiers. We can quickly see that some characters appear with different names 
# (first name only or full name). Cleaning the data!

False


In [357]:
names = the_boys[the_boys.groupby(['nconst'])['character'].transform('nunique') > 1]['character'].unique().tolist()
#names  = the_boys_new['character'].unique().tolist()
for x in sorted(names):
    print(x)

# So we see that one of the main characters, Hughie Campbell, appears under different names - Hughie, Hughie Campbell

the_boys = the_boys.replace(['Hugh Campbell', 'Hughie'], 'Hughie Campbell')
the_boys = the_boys.replace(['Butcher'], 'Billy Butcher')
the_boys = the_boys.replace(['Kimiko'], 'Kimiko Miyashiro')
the_boys = the_boys.replace(['Mother’s Milk'], 'Mother\'s Milk')
the_boys = the_boys.replace(['Young Black Noir'], 'Black Noir')
#the_boys = the_boys.replace(['Ryan'], 'Ryan Butcher')

Alastair (movie version)
Billy Butcher
Billy Zane
Board Member
Butcher
Crimson Countess
Eagle Soldier Boy
Entertainment Reporter
Female News Anchor
Fox Crimson Countess
Horse Tessa
Horse Tommy
Hugh Campbell
Hughie
Hughie Campbell
Hughie's Father
Interviewer
Kimiko
Kimiko Miyashiro
Lamplighter
Maureen
Meerkat Stan Edgar
Mother's Milk
Mother’s Milk
NNC News Anchor
Nurse
Random dude
Robert Singer
Ryan
Ryan Butcher
Sam
Sam Butcher
Secretary of Defense Robert A. Singer
Seth
Seth Reed
Seth Rogen
Sheep Black Noir
SirCumsALot779
Soldier Boy
Stan Edgar
Susan Rayner
Susan Raynor
Teen
Teen Lenny
Tessa
The Female
Tommy
Tony
Young Black Noir
Zoe
Zoe Neuman


In [358]:
# Character Names column is clean now!
character_names_clean = the_boys['character'].unique()
len(character_names_clean)

298

In [359]:
# What are the most popular characters over all season based on their screen time?
character_duration = []

for character in character_names_clean:
  character_duration.append(sum(the_boys[the_boys['character'] == character]['Scene Duration'].tolist())/3600000)

most_appeared_characters_by_duration = {character_names_clean[i]: character_duration[i] for i in range(len(character_names_clean))}
most_appeared_characters_by_duration = dict(sorted(most_appeared_characters_by_duration.items(), key=lambda item: item[1], reverse = True))

characters_by_duration = pd.DataFrame(most_appeared_characters_by_duration.items())
characters_by_duration.columns = ['Character Name', 'Total screen time (hours)']
characters_by_duration.head(10)

,Character Name,Total screen time (hours)
0,Hughie Campbell,8.069167
1,Billy Butcher,7.456389
2,Starlight,5.694444
3,Homelander,5.675556
4,Mother's Milk,5.326111
5,Frenchie,5.139722
6,Kimiko Miyashiro,2.810556
7,Queen Maeve,2.264722
8,A-Train,2.222500
9,Ashley Barrett,2.058611


In [360]:
# Let's find out more data!-
# Nr chracters by season
# Top characters by season
# Screen time fluctuation over season
# Are there characters who died?

In [361]:
# Season 1
dummy1 = the_boys[the_boys['Season'] == '1']
aggregation_functions = {'Scene Duration': 'sum', 'nconst': 'first'}
df_season1 = dummy1.groupby(dummy1['character']).aggregate(aggregation_functions).sort_values(by = 'Scene Duration', ascending=False)
df_season1['Scene Duration'] = df_season1['Scene Duration']/3600000
df_season1 = df_season1.reset_index()
df_season1

,character,Scene Duration,nconst
0,Hughie Campbell,3.674167,nm4425051
1,Billy Butcher,2.934722,nm0881631
2,Frenchie,2.074722,nm6150071
3,Starlight,1.823056,nm3929195
4,Mother's Milk,1.740000,nm0022306
...,...,...,...
121,Commercial Mom,0.005833,nm6844613
122,Tina,0.005833,nm0715250
123,Clerk,0.005556,nm4981734
124,Female from the Car,0.003056,nm2854706


In [362]:
character_names_s1 = df_season1['nconst'].nunique()
print("Season 1 has {} characters".format(character_names_s1))

Season 1 has 126 characters


In [363]:
# Top 10 characters by number of appearances in season 1
most_times_appeared_s1 = dummy1.character.value_counts()
most_times_appeared_s1_df = pd.DataFrame(most_times_appeared_s1)
most_times_appeared_s1_df = most_times_appeared_s1_df.rename(columns={'character': 'Number of Appearances'})
most_times_appeared_s1_df[:10]

,Number of Appearances
Hughie Campbell,74
Billy Butcher,65
Frenchie,44
Starlight,42
Homelander,42
Mother's Milk,38
Madelyn Stillwell,34
A-Train,27
The Deep,24
Queen Maeve,23


In [364]:
#Season 2
dummy2 = the_boys[the_boys['Season'] == '2']
df_season2 = dummy2.groupby(dummy2['character']).aggregate(aggregation_functions).sort_values(by = 'Scene Duration', ascending=False)
df_season2['Scene Duration'] = df_season2['Scene Duration']/3600000
df_season2 = df_season2.reset_index()
df_season2

,character,Scene Duration,nconst
0,Hughie Campbell,2.371389,nm4425051
1,Billy Butcher,2.298056,nm0881631
2,Mother's Milk,2.204444,nm0022306
3,Homelander,2.103056,nm1102278
4,Starlight,2.102500,nm3929195
...,...,...,...
103,Super Villain,0.007222,nm5318612
104,Elizabeth,0.007222,nm6596476
105,Interviewer,0.006111,nm10651052
106,Michel,0.005556,nm3997201


In [365]:
character_names_s2 = df_season2['nconst'].nunique()
print("Season 2 has {} characters".format(character_names_s2))

Season 2 has 104 characters


In [366]:
# Top 10 characters by number of appearances in season 2
most_times_appeared_s2 = dummy2.character.value_counts()
most_times_appeared_s2_df = pd.DataFrame(most_times_appeared_s2)
most_times_appeared_s2_df = most_times_appeared_s2_df.rename(columns={'character': 'Number of Appearances'})
most_times_appeared_s2_df[:10]

,Number of Appearances
Hughie Campbell,64
Billy Butcher,58
Starlight,57
Homelander,56
Mother's Milk,54
Frenchie,50
Kimiko Miyashiro,44
Stormfront,43
Queen Maeve,25
Ashley Barrett,21


In [367]:
#Season 3
dummy3 = the_boys[the_boys['Season'] == '3']
df_season3 = dummy3.groupby(dummy3['character']).aggregate(aggregation_functions).sort_values(by = 'Scene Duration', ascending=False)
df_season3['Scene Duration'] = df_season3['Scene Duration']/3600000
df_season3 = df_season3.reset_index()
df_season3

,character,Scene Duration,nconst
0,Billy Butcher,2.223611,nm0881631
1,Hughie Campbell,2.023611,nm4425051
2,Homelander,1.846111,nm1102278
3,Starlight,1.768889,nm3929195
4,Mother's Milk,1.381667,nm0022306
...,...,...,...
123,Butcher's Mother,0.003611,nm5531364
124,Teen,0.003611,nm5385972
125,Guy,0.002778,nm5045671
126,Board Member,0.001389,nm1178525


In [368]:
character_names_s3 = df_season3['nconst'].nunique()
print("Season 3 has {} characters".format(character_names_s3))

Season 3 has 117 characters


In [369]:
# Top 10 characters by number of appearances in season 3
most_times_appeared_s3 = dummy3.character.value_counts()
most_times_appeared_s3_df = pd.DataFrame(most_times_appeared_s3)
most_times_appeared_s3_df = most_times_appeared_s3_df.rename(columns={'character': 'Number of Appearances'})
most_times_appeared_s3_df[:10]

,Number of Appearances
Billy Butcher,130
Hughie Campbell,125
Starlight,109
Homelander,103
Mother's Milk,91
Soldier Boy,70
Kimiko Miyashiro,63
Frenchie,61
Ashley Barrett,49
Queen Maeve,35


In [370]:
# How has the screen time of main characters fluctuate through seasons?
df_season1 = df_season1.sort_values(by=['nconst'])
df_season2 = df_season2.sort_values(by=['nconst'])
df_season3 = df_season3.sort_values(by=['nconst'])

# Between s2 and 1
between_s2_and_s1  = pd.DataFrame(df_season2['Scene Duration'] - df_season1['Scene Duration'])
between_s2_and_s1['character'] = df_season1['character']
between_s2_and_s1 = between_s2_and_s1.rename(columns={'Scene Duration': 'Duration Change'})
between_s2_and_s1 = between_s2_and_s1.loc[:, ['character', 'Duration Change']]
between_s2_and_s1.head(10)

,character,Duration Change
0,Hughie Campbell,-1.302778
1,Billy Butcher,-0.636667
2,Frenchie,0.129722
3,Starlight,0.280000
4,Mother's Milk,0.362500
5,Homelander,0.148889
6,Madelyn Stillwell,0.507222
7,A-Train,0.462778
8,Queen Maeve,-0.092500
9,The Female,0.005556


In [371]:
# Between s2 and 3
between_s2_and_s3  = pd.DataFrame(df_season3['Scene Duration'] - df_season2['Scene Duration'])
between_s2_and_s3['character'] = df_season2['character']
between_s2_and_s3 = between_s2_and_s3.rename(columns={'Scene Duration': 'Duration Change'})
between_s2_and_s3 = between_s2_and_s3.loc[:, ['character', 'Duration Change']]
between_s2_and_s3.head(10)

,character,Duration Change
0,Hughie Campbell,-0.147778
1,Billy Butcher,-0.274444
2,Mother's Milk,-0.358333
3,Homelander,-0.334167
4,Starlight,-0.720833
5,Frenchie,-0.685556
6,Kimiko Miyashiro,-0.676667
7,Stormfront,-0.612778
8,Queen Maeve,-0.066667
9,Becca Butcher,-0.167778


In [373]:
dead = df_season1[df_season1['character'].apply(lambda x: not df_season3['character'].str.contains(x).any())]
dead = dead.sort_values(by=['Scene Duration'], ascending=False)
dead.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,character,Scene Duration,nconst
6,Madelyn Stillwell,1.236389,nm0000223
9,The Female,0.763611,nm7232332
12,Popclaw,0.531944,nm0020329
15,Translucent,0.417500,nm1307435
16,Shockwave,0.382500,nm5973363
17,Ezekiel,0.335556,nm0072648
19,Susan Raynor,0.252500,nm0261170
20,Mesmer,0.247500,nm0005286
21,Becca Butcher,0.164722,nm1416900
24,Gary,0.137500,nm0609150
